In [1]:
# Разархивируем данные
!tar -xvf 'internship_data.tar.gz'

Выходные данные были обрезаны до нескольких последних строк (5000).
internship_data/female/176541.jpg
internship_data/female/197816.jpg
internship_data/female/021766.jpg
internship_data/female/023116.jpg
internship_data/female/180754.jpg
internship_data/female/049526.jpg
internship_data/female/162901.jpg
internship_data/female/178184.jpg
internship_data/female/000493.jpg
internship_data/female/117257.jpg
internship_data/female/146389.jpg
internship_data/female/097102.jpg
internship_data/female/162119.jpg
internship_data/female/147236.jpg
internship_data/female/123583.jpg
internship_data/female/082756.jpg
internship_data/female/170656.jpg
internship_data/female/098709.jpg
internship_data/female/012255.jpg
internship_data/female/085090.jpg
internship_data/female/069812.jpg
internship_data/female/024874.jpg
internship_data/female/107100.jpg
internship_data/female/153792.jpg
internship_data/female/092234.jpg
internship_data/female/176085.jpg
internship_data/female/079875.jpg
internship_dat

In [3]:
!pip install pytorch-lightning

     |████████████████████████████████| 409kB 4.7MB/s 
     |████████████████████████████████| 829kB 11.8MB/s 
     |████████████████████████████████| 276kB 27.8MB/s 
     |████████████████████████████████| 2.8MB 29.7MB/s 
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=547efe3f713265c71db861a696d1d20cbb9ea011bee3e05be083469203c55051
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=caa9be37f3cf3071be2feab1c1588bfa7d550fb36570b24ada0f03ba9357c6c5
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built future PyYAML
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 2.2.0 which is incompatible.
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully 

In [4]:
!pip install efficientnet-pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp36-none-any.whl size=16031 sha256=702c91801b39d7e686c8d6b273f954423b78db091465360fd2a3058c4ba4ada6
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch


In [5]:
import matplotlib.pyplot as plt
import os
import numpy as np

import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder


from albumentations.core.transforms_interface import DualTransform
from albumentations.augmentations import functional as F
 
import albumentations as A
import albumentations.pytorch as AP


import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.metrics.classification import Accuracy
from pytorch_lightning.callbacks import ModelCheckpoint

from sklearn.metrics import accuracy_score

from efficientnet_pytorch import EfficientNet
from sklearn.model_selection import train_test_split
import os, cv2
from shutil import move

In [7]:
np.random.seed(42)

In [8]:
# создаю тренировочный и валидационный датасет

!mkdir train
!mkdir train/male
!mkdir train/female

!mkdir val
!mkdir val/male
!mkdir val/female

In [9]:
# функция для перемещения файлов files из директории scr в dst

def mv_to(files, scr, dst):
  for f in files:
    move(os.path.join(scr, f), os.path.join(dst,f))

In [10]:
#разбиение на треин и тест

val_folder = '/content/val'
train_folder = '/content/train'

path_male = '/content/internship_data/male'
path_female = '/content/internship_data/female'

files_male = os.listdir(path_male) 
files_female = os.listdir(path_female)

tr_male, tst_male = train_test_split(files_male, test_size=0.2, random_state = 42)
tr_female, tst_female = train_test_split(files_female, test_size=0.2, random_state = 43)

mv_to(tr_male, path_male, os.path.join(train_folder, 'male'))
mv_to(tst_male, path_male, os.path.join(val_folder, 'male'))

mv_to(tr_female, path_female, os.path.join(train_folder, 'female'))
mv_to(tst_female, path_female, os.path.join(val_folder, 'female'))

In [11]:
# определяю трансформацию, которая сохраняет соотношение сторон картинки и изменяет размер наибольшей стороны

class max_dim_resize(DualTransform):
    def __init__(self, max_size , interpolation=cv2.INTER_LINEAR, always_apply=False, p=1):
        super(max_dim_resize, self).__init__(always_apply, p)
        self.max_size = max_size
        self.interpolation = interpolation
 
    def apply(self, img, interpolation=cv2.INTER_LINEAR, **params):
        a, b, *_ = img.shape
        alpha = float(self.max_size)/max(a,b)
        n_height, n_width = int(a*alpha), int(b*alpha)
        return F.resize(img, height=n_height, width=n_width, interpolation=interpolation)
 
    def apply_to_bbox(self, bbox, **params):
        # Bounding box coordinates are scale invariant
        return bbox
 
    def apply_to_keypoint(self, keypoint, **params):
        height = params["rows"]
        width = params["cols"]
        scale_x = self.width / width
        scale_y = self.height / height
        return F.keypoint_scale(keypoint, scale_x, scale_y)
 
    def get_transform_init_args_names(self):
        return ("height", "width", "interpolation")

In [12]:
#определяю трансформации

norm_dict = {
    'mean' : np.array([0.485, 0.456, 0.406]),
    'std' : np.array([0.229, 0.224, 0.225])
}
 
train_transforms = A.Compose([
    A.HorizontalFlip(),
    A.Rotate(10),
    max_dim_resize(224),
    A.PadIfNeeded(224,224, border_mode=0),
    AP.transforms.ToTensor(normalize=norm_dict)
])
 
val_transforms = A.Compose([
    max_dim_resize(224),
    A.PadIfNeeded(224,224, border_mode=0),
    AP.transforms.ToTensor(normalize=norm_dict)
])

In [13]:
# класс для совместимости ImageFolder в pytorch и albumentations 

class kostil():
  def __init__(self, trans):
    self.trans = trans
  def __call__(self, im):
    transformed = self.trans(image = np.array(im))
    return transformed['image']

In [14]:
# класс для предтренировки весов с помощью автоэнкодера

class autoencoder_pretrain(pl.LightningModule):

  def __init__(self, lr = 1e-4):
    super().__init__()
    self.model = EfficientNet.from_pretrained('efficientnet-b4')
    # меняю последний слой
    self.model._fc = nn.Identity()
    self.encoder = nn.Sequential(
        nn.ConvTranspose2d(1792, 1024, 5, 2),
        nn.LeakyReLU(0.2),
        nn.BatchNorm2d(1024),
        nn.ConvTranspose2d(1024, 512, 5,2),
        nn.LeakyReLU(0.2),
        nn.BatchNorm2d(512),
        nn.ConvTranspose2d(512, 256, 3,2),
        nn.LeakyReLU(0.2),
        nn.BatchNorm2d(256),
        nn.ConvTranspose2d(256, 128, 3,2),
        nn.LeakyReLU(0.2),
        nn.BatchNorm2d(128),
        nn.ConvTranspose2d(128,64, 3,2),
        nn.LeakyReLU(0.2),
        nn.BatchNorm2d(64),
        nn.ConvTranspose2d(64, 3, 4,2)
    )

    # определяю датасеты
    self.train_set = ImageFolder(train_folder,transform=kostil(train_transforms))
    self.val_set = ImageFolder(val_folder,transform=kostil(val_transforms))

    self.loss = nn.MSELoss()
   

    self.lr = lr

  def forward(self, x):
    latent = self.model(x).unsqueeze(0).unsqueeze(0).permute(2,3,0,1)
    return self.encoder(latent)
  
  def training_step(self, batch, batch_nb):
    x, y = batch
    yh = self.forward(x)
    loss = self.loss(yh, x)

    result = pl.TrainResult(loss)
    
    log_dict = {'train_loss': loss,}

    result.log_dict(log_dict,prog_bar=True, on_epoch=True)
    return result
  
  def validation_step(self, batch, batch_nb):
    x, y = batch
    yh = self.forward(x)
    loss = self.loss(yh, x)

    result = pl.EvalResult(checkpoint_on = loss)
    log_dict = {'val_loss': loss}

    result.log_dict(log_dict,prog_bar=True, on_epoch=True)
    return result

  def train_dataloader(self):
    return DataLoader(self.train_set, 32, shuffle = True)

  def val_dataloader(self):
    return DataLoader(self.val_set, 32, shuffle = True)

  
  def configure_optimizers(self):
    opt = optim.Adam(self.parameters(), lr = self.lr)
    shedul = optim.lr_scheduler.StepLR(opt, 7)
    return [opt], [shedul]

In [17]:
# классификатор

class classifier(pl.LightningModule):

  def __init__(self, lr = 1e-4):
    super().__init__()
    self.model = EfficientNet.from_pretrained('efficientnet-b4')
    # меняю последний слой
    self.model._fc = nn.Identity()
    self.fc = nn.Linear(1792, 1)
    # определяю датасеты
    self.train_set = ImageFolder(train_folder,transform=kostil(train_transforms))
    self.val_set = ImageFolder(val_folder,transform=kostil(val_transforms))

    self.loss = nn.BCEWithLogitsLoss()
    self.metric = Accuracy(num_classes=2)

    self.lr = lr

  def forward(self, x):
    return self.fc(self.model(x))
  
  def training_step(self, batch, batch_nb):
    x, y = batch
    yh = self.forward(x).squeeze()
    loss = self.loss(yh, y.type_as(yh))

    result = pl.TrainResult(loss)
    
    log_dict = {'train_loss': loss,
                'train_acc':self.metric((torch.sigmoid(yh)>0.5).int(), y)}

    result.log_dict(log_dict,prog_bar=True, on_epoch=True)
    return result
  
  def validation_step(self, batch, batch_nb):
    x, y = batch
    yh = self.forward(x).squeeze()
    loss = self.loss(yh, y.type_as(yh))

    result = pl.EvalResult(checkpoint_on = loss)
    log_dict = {'val_loss': loss,
                'val_acc':self.metric((torch.sigmoid(yh)>0.5).int(), y)}

    result.log_dict(log_dict,prog_bar=True, on_epoch=True)
    return result

  def train_dataloader(self):
    return DataLoader(self.train_set, 32, shuffle = True)

  def val_dataloader(self):
    return DataLoader(self.val_set, 32, shuffle = True)

  
  def configure_optimizers(self):
    opt = optim.Adam(self.parameters(), lr = self.lr)
    shedul = optim.lr_scheduler.StepLR(opt, 7)
    return [opt], [shedul]

In [1]:
def train_model(model, epochs):
    checkpoint_callback = ModelCheckpoint(
    save_top_k=1,
    filepath = os.path.abspath(os.getcwd()),
    verbose=True,
    save_last = True,
    monitor='val_loss',
    mode='min',
    prefix=''
    )
    
    trainer = pl.Trainer(gpus = 1,
                    max_epochs = epochs,
                    default_root_dir = os.path.abspath(os.getcwd()),
                    checkpoint_callback = checkpoint_callback,
                     limit_train_batches=2500,
                     limit_val_batches=1.0,
                     auto_lr_find = True,
                     gradient_clip_val=0.5,
                     )
    
    trainer.fit(model)

In [18]:
model = classifier()
md_aut = autoencoder_pretrain()

Loaded pretrained weights for efficientnet-b4


In [ ]:
train_model(md_aut, 10)

In [38]:
import copy

In [39]:
model.model = copy.deepcopy(md_aut.model)

In [ ]:
train_model(model, 15)